In [5]:
import requests
import json
from lxml import etree
import pandas as pd
import re
import math

##调用getMerchantComment接口获取评论json
def getShopPage(shopID,offSet):
    simulateChromeBrowserData = {
        'Accept':'*/*',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive',
        'Host':'sz.meituan.com',
        'Referer':'http://wh.meituan.com/meishi/',
        # 'Cookie': '_lxsdk_cuid=16212a00d8ec8-07cdb6596bad8e-178123e-1fa400-16212a00d8fc8; lat=22.780886; lng=113.906362; client-id=34908f62-ea11-4211-b60a-f62c32288b2e; uuid=9be4f96971ac4c9cab4c.1520730903.1.0.0; webloc_geo=22.527181%2C113.938582%2Cwgs84; ci=30; _lxsdk=16212a1cddfc8-011369480302e7-178123e-1fa400-16212a1cddfc8; __mta=247430459.1520730902128.1520731016684.1520731025187.5; _lxsdk_s=16212a00d8f-c83-6e-376%7C%7C9',
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'
    }
    purl='https://www.meituan.com/meishi/api/poi/getMerchantComment'
    params = {
        'id' : shopID,
        'userId' : 0,
        'offset' : offSet*10,#偏移量，也是页码信息
        'pageSize' : 10,
        'sortType' : 1
    }
    page = requests.get(purl,params=params,headers=simulateChromeBrowserData)
    return page



In [6]:
def dataClean(comment):  #只保留中文
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    commentc = re.sub(pattern, '', comment)
    return commentc

def parse_POI_Shop_Json(shop_json):
    one_comment_data = pd.DataFrame()
    if 'data'in shop_json:
        for eachShopInfo in shop_json.get('data').get('comments'):
            Comment = dataClean(eachShopInfo.get('comment'))
            if not Comment:
                continue
            IsGreet = 1 if eachShopInfo.get('star')>= 30 else 0
            one_comment_data = one_comment_data.append(pd.DataFrame({'comment':[Comment],'sentiment':[IsGreet]}))
    return one_comment_data

In [7]:
Comment_datas = pd.DataFrame();
# allCommentNum商家的最大评论数
allCommentNum=20000
for i in range(allCommentNum//10):
    page = getShopPage(74779,i)
    #print(page.url)
    # print(page.json())
    Comment_data = parse_POI_Shop_Json(page.json())
    Comment_datas = Comment_datas.append(Comment_data)
print(Comment_datas)
Comment_datas.to_csv('CommentOfSentiment_datas.csv', index=False)


                                              comment  sentiment
0                                个人吃加了个羊腿不错的一家西餐厅还会光顾          1
0   我和男朋友是秀玉的老顾客了第一次认识就是在秀玉吃的饭他们家的环境优雅菜品味道也挺不错今天和男...          1
0   每次来秀玉觉得心情很好因为环境不错东西又很好吃因为没有拍照的习惯只能拿朋友做的东西来凑了貌似...          1
0   和室友一起来的这里位置很好找整个装饰也很有格调适合人们休息点了香辣咖喱鸡焗饭冰淇淋水果圣代奶...          1
0                                  还不错吧环境优雅菜品种类多味道还可以          1
..                                                ...        ...
0                            吃的牛排披萨很饱味道不错有点小贵再优惠点就更好了          1
0   好非常好环境好东西好吃跟同学来的同学也很喜欢我超级喜欢这个地方环境高端食物大气真是上档次很喜...          1
0   菜的口味好好好好好好好好好好好好好好好好好好好好好好好好好好好好好好了吗呢吧主题库里好好好好...          1
0                        牛排煎的太过了上菜慢有可能是人多味道没有魅力船说餐吧的好          1
0   我們是下午去的燈光優美環境不錯服務員也很熱情沒有因為是團購就怠慢我們上菜的速度也超快的份量不...          1

[19971 rows x 2 columns]
